# Projeto final: criar um projeto de ML de dados tabulares de ponta a ponta usando o SageMaker Studio e o SDK Python do Amazon SageMaker Python

## Configuração do ambiente

Este código de configuração básico foi incluído para ajudar você a começar. Leia e execute essas células primeiro para instalar os pacotes e criar as variáveis.

In [ ]:
%%capture
%pip install -U shap
%pip install -U smdebug
%pip install imbalanced-learn
%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install sagemaker
%pip install -U seaborn

In [ ]:
#required-libraries

import boto3
import datetime as datetime
import io
import IPython
import json
import math
import matplotlib.pyplot as plt  # visualization
import numpy as np
import pandas as pd
import pathlib
import re
import sagemaker
import seaborn as sns  # visualization
import statistics
import string
import sys
import time
import zipfile

from imblearn.over_sampling import SMOTE

from sagemaker import clarify
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition
from sagemaker.debugger import CollectionConfig, DebuggerHookConfig, FrameworkProfile, ProfilerConfig, ProfilerRule, Rule, rule_configs
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TransformInput
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.network import NetworkConfig
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.steps import TuningStep
from sagemaker.xgboost.estimator import XGBoost
from sklearn.model_selection import train_test_split

In [ ]:
#basic-variable-code-and-settings

%matplotlib inline
base_job_name = "capstone-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "sagemaker/capstone"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
save_interval = 5
sns.set(color_codes=True)

## Descrição do conjunto de dados

Cinco tabelas são armazenados em um bucket do Amazon Simple Storage Service (Amazon S3):
- **claims.csv**: uma tabela contendo dados brutos de sinistros.
- **customers.csv**: um tabela contendo dados brutos de clientes.
- **claims_preprocessed.csv**: uma tabela contendo dados processados de sinistros.
- **customers_preprocessed.csv**: uma tabela contendo dados processados de clientes.
- **claims_customer.csv**: uma tabela unificada no **policy_id** com os dados processados de sinistros e clientes.

Neste laboratório, comece com as tabelas **claims.csv** e **customers.csv**. Você vai processá-las no **Desafio 1** usando o Amazon SageMaker Data Wrangler. Se você tiver dúvidas ou quiser uma referência de como o conjunto de dados deve ser, examine as tabelas pré-processadas.

Para este conjunto de dados, o destino é **fraud**, uma coluna na tabela de sinistros.

A tabela de sinistros inclui os seguintes campos: 

- **policy_id**: o ID exclusivo da política.
- **driver_relationship**: uma lista de relacionamentos (Spouse, Self, Child, Other, N/A).
- **incident_type**: o tipo de incidente relatado (Break-In, Collision, Theft).
- **collision_type**: o local da colisão (Front, Rear, Side, N/A).
- **incident_severity**: a severidade do incidente (Minor, Major, Totaled).
- **authorities_contacted**: o tipo de autoridade do primeiro contato (None, Police, Ambulance, Fire).
- **num_vehicles_involved**: o número de veículos envolvidos no incidente (um intervalo de 1 a 6).
- **num_injuries**: o número de ferimentos envolvidos no incidente (um intervalo de 1 a 4).
- **num_witnesses**: o número de testemunhas do incidente (um intervalo de 1 a 5).
- **police_report_available**: se há um relatório policial envolvido (yes ou no).
- **injury_claim**: o valor reivindicado, relativo a ferimentos, em dólares americanos (300 a 576300 USD).
- **injury_claim**: o valor reivindicado, relativo a danos no veículo, em dólares americanos (1000 a 51051 USD).
- **total_claim_amount**: o valor total reivindicado para ferimentos e danos (2100 a 588868 USD).
- **incident_month**: o mês do incidente (um intervalo de 1 a 12).
- **incident_day**: o dia do incidente (um intervalo de 1 a 31).
- **incident_dow**: o dia da semana do incidente (um intervalo de 0 a 6 representando de sábado a domingo).
- **incident_hour**: o horário do incidente (um intervalo de 0 a 23)
- **fraud**: se a apólice era fraudulenta (0 ou 1).

A tabela de clientes inclui os seguintes campos:

- **policy_id**: o ID exclusivo da política.
- **customer_age**: a idade do cliente (um intervalo de 18 a 70).
- **months_as_customer**: o número de meses em que esse cliente já pagou o seguro (um intervalo de 1 a 495).
- **num_claims_past_year**: o número de sinistros registrados pelo cliente no ano passado.
- **num_insurers_past_5_years**: o número de seguradoras que o cliente teve nos últimos cinco anos.
- **policy_state**: o estado dos EUA em que o cliente vive (AZ, CA, ID, NV, OR, WA).
- **policy_deductable**: o valor deduzível da apólice em dólares americanos (um intervalo de 750 a 1100 USD).
- **policy_annual_premium**: o prêmio anual da apólice em dólares americanos (um intervalo de 2200 a 3000 USD).
- **policy_liability**: o máximo de responsabilização por ferimentos corporais, dividido em ferimentos individuais ou no corpo todo (15/30, 25/50, 60/90, 100/200).
- **customer_zip**: o CEP do cliente (um intervalo de 83201 a 99362).
- **customer_gender**: o gênero do cliente (Male, Female, Other, Unknown).
- **customer_education**: o nível educacional do cliente (Below High School, High School, Associate, Bachelor, Advanced Degree).
- **auto_year**: o ano em que o automóvel foi fabricado (um intervalo de 2001 a 2020).

É possível unificar as tabelas com uma unificação interna na coluna **policy_id**.

## Navegação no laboratório de desafio

Este laboratório está configurado com links que navegam entre as tarefas de desafio e o apêndice no final do notebook. Se você quiser examinar um item no apêndice, escolha o hiperlink associado. Para retornar ao desafio em que você está trabalhando no momento, escolha o hiperlink da tarefa correspondente no apêndice.

O laboratório é organizado desta forma:

- Desafio 1: Analisar e preparar o conjunto de dados com o SageMaker Data Wrangler
- Desafio 2: Criar grupos de recursos no SageMaker Feature Store
- Desafio 3: Treinar o modelo
- Desafio 4: Avaliar o viés do modelo
- Desafio 5: Transformação em lote
- Desafio 6: Criar um pipeline automatizado
- Apêndice

## Desafio 1: Analisar e preparar o conjunto de dados com o SageMaker Data Wrangler

A AnyCompany Consulting recebeu uma solicitação para analisar conjuntos de dados de fraude em seguro de automóvel e criar um modelo para ajudar a prever a probabilidade de que as novos sinistros sejam fraudulentos ou não. Há 5 mil registros de cliente. Cada sinistro está rotulado como fraudulento ou não. Você pode usar esses dados para treinar, testar e validar o modelo antes de executar a inferência em uma nova coleção de registros em lote.

Use os recursos de análise do Amazon SageMaker Data Wrangler para visualizar as distribuições dos dados em colunas importantes, verificar a correlação entre as colunas e conferir o vazamento de dados de destino. Em seguida, crie um modelo de linha de base rápido. Depois, use os recursos de processamento do SageMaker Data Wrangler a fim de transformar as colunas adequando-as para treinar um modelo com melhor desempenho. 

Para concluir essa tarefa, realize estas subtarefas:

- Examinar os dados.
- Concluir uma análise exploratória de dados no Amazon SageMaker Studio.
- Usar um trabalho de processador do Amazon SageMaker Clarify para gerar um relatório de viés.
- Preparar os dados.

Este desafio exige cerca de *100* minutos para ser concluído.

### Tarefa 1.1: Examinar os dados

<a id="task1-1-continue"></a>

Acesse o conjunto de dados de seguro de automóvel tabular armazenado no repositório e examine uma amostra do conjunto de dados. O repositório contém duas tabelas não processadas. Uma contém dados de clientes, chamada **customers.csv**, e a outra contém dados de sinistros, chamada **claims.csv**.

**Dica 1**: as tabelas não processadas estão localizadas na pasta **./data/**.

**Dica 2**: as tabelas **claims.csv** e **customers.csv** são as tabelas não processadas.

Reserve um momento para explorar as tabelas. Há campos que se destacam? Há campos que exigem um pré-processamento cuidadoso?

Para obter etapas detalhadas que mostram como examinar os dados, consulte <a href="#task1-1" target="_self">**Examinar os dados (Tarefa 1.1)**</a> na seção *Apêndice*.

Depois de acessar a tabela de fraudes em seguro de automóvel e examinar uma amostra do conjunto de dados, você terá concluído esta tarefa.

In [ ]:
#add_your_task_1_1_code_here



### Tarefa 1.2: Realizar uma análise exploratória de dados no SageMaker Studio 

<a id="task1-2-continue"></a>

Faça uma análise exploratória de dados examinando os dados, identificando possíveis problemas no conjunto de dados e verificando se há correlações fortes entre as colunas e o destino. Você pode explorar os dados no SageMaker Data Wrangler e no notebook.

Especificamente, dedique um tempo para examinar os seguintes itens:
- **Histogramas de coluna**: examine as colunas em um formato visual e verifique quais tipos de valores estão no conjunto de dados.
- **Modelo rápido**: examine o conjunto de dados e pense qual seria o resultado esperado do modelo.
- **Correlação de recursos**: verifique se existe uma correlação forte entre as colunas e o destino.
- **Vazamento de dados de destino**: verifique se há dados que dependem do valor de destino.

Ao ser aberto, o SageMaker Data Wrangler abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **capstone.ipynb** para a lateral ou selecione (clique com o botão direito) a aba **capstone.ipynb** e escolha **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis durante o trabalho no fluxo do SageMaker Data Wrangler.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Quando terminar de explorar as etapas do SageMaker Data Wrangler, retorne ao notebook selecionando a aba **capstone.ipynb**.

Se aparecer o erro **"O seguinte tipo de instância não está disponível: ml.m5.4xlarge. Tente selecionar uma instância diferente abaixo."** durante a criação de um arquivo de fluxo, escolha outro tipo de instância. Tente o **ml.m5.8xlarge** em seguida.

Se a mensagem **Ocorreu um erro ao carregar esta visualização** for exibida, feche a aba **untitled.flow** e reabra o arquivo de fluxo usando o navegador de arquivo.

**Dica 1**: há muitas maneiras de explorar o conjunto de dados. Abra um fluxo do SageMaker Data Wrangler para começar. É necessário importar o **claims.csv** e o **customers.csv** no SageMaker Data Wrangler por meio do bucket do S3 que contém **databucket-** no nome.

**Dica 2**: para importar uma segunda tabela, retorne ao **Fuxo de dados**, selecione a aba **Import** (Importar) para importar outro conjunto de dados.

**Dica 3**: **Obter informações de dados** e **Adicionar análise** são duas maneiras de explorar os dados no SageMaker Data Wrangler. Depois de examinar alguns gráficos de amostra dos dados, você poderá usar outras ferramentas de criação de gráficos no notebook para analisar os dados, se desejar. As bibliotecas **plt** e **sns** foram instaladas. Fique à vontade para usar qualquer ferramenta de análise que você já conheça para explorar o conjunto de dados.  

**Dica 4**: tente unificar as duas tabelas usando **Join** (Unificar) em **policy_id**. Depois, gere outro relatório de informações. Você pode usar uma unificação **Interna** para essas tabelas.

Você obteve resultados mais significativos com um conjunto de dados unificado?

Para obter etapas detalhadas que mostram como explorar um conjunto de dados no SageMaker Studio, consulte <a href="#task1-2-1" target="_self">**Explorar um conjunto de dados no SageMaker Studio (Tarefa 1.2)**</a> na seção *Apêndice*.

Para obter etapas detalhadas que mostram como explorar um conjunto de dados no notebook, consulte <a href="#task1-2-2" target="_self">**Explorar um conjunto de dados no notebook (Tarefa 1.2)**</a> na seção *Apêndice*.

Depois de processar os dados usando o SageMaker Data Wrangler, explorar o conjunto de dados e identificar as etapas de processamento que deseja realizar, você terá concluído esta tarefa.

In [ ]:
#add_your_task_1_2_code_here



### Tarefa 1.3: Usar um trabalho de processador do SageMaker Clarify para gerar um relatório de viés

<a id="task1-3-continue"></a>

Use o SageMaker Clarify para gerar um relatório de viés de pré-treinamento para capturar o desequilíbrio de classes nos dados. Use um fluxo do SageMaker Data Wrangler para gerar o relatório de viés no SageMaker Studio.
1. Comece unificando as duas tabelas.

- Unificar: unificação **interna** de **claims.csv** com **customers.csv** em **policy_id**.

**Dica 1**: para criar um relatório de viés de pré-treinamento, adicione uma nova análise ao fluxo do SageMaker Data Wrangler e escolha **Bias Report** (Relatório de viés) em **Analysis type** (Tipo de análise).

**Dica 2**: é possível gerar o relatório de viés várias vezes escolhendo recursos diferentes para analisar a cada vez.

Para obter etapas detalhadas que mostram como unificar tabelas com o SageMaker Data Wrangler, consulte <a href="#task1-3-1" target="_self">**Unificar tabelas no SageMaker Studio (Tarefa 1.3)**</a> na seção *Apêndice*.

Para obter etapas detalhadas que mostram como gerar um relatório de viés de pré-treinamento, consulte <a href="#task1-3-2" target="_self">**Gerar um relatório de viés de pré-treinamento (Tarefa 1.3)**</a> na seção *Apêndice*.

Depois de gerar o relatório de viés de pré-treinamento e visualizar o relatório, você terá concluído esta tarefa.

### Tarefa 1.4: Preparar os dados

<a id="task1-4-continue"></a>

Prepare o conjunto de dados usando o SageMaker Data Wrangler. Concentre-se nas seguintes transformações, mas fique à vontade para incluir outras:

- Codificar de maneira categórica (codificação one-hot): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type** e **policy_state**.
- Codificar de maneira ordinal: **customer_education**, **policy_liability**, **incident_severity** e **police_report_available**.
- Analisar coluna como tipo: **vehicle_claim** e **total_claim_amount** de **Float** para **Long**.
- Gerenciar colunas (Remover coluna): **customer_zip**.
- Gerenciar colunas (Mover coluna): **fraud** (usando **Mover para início**).
- Gerenciar colunas (Renomear coluna): remova o símbolo **/** de **collision_type_N/A** e **driver_relationship_N/A** com **_**.
- Gerenciar colunas (Renomear coluna): renomeie **policy_id_0** como **policy_id**.

**Dica 1**: unifique a tabela **claims** às tabelas **customers** usando uma unificação no SageMaker Data Wrangler. 

**Dica 2**: unifique as duas tabelas na coluna **policy_id**.

**Dica 3**: adicione transformações usando a opção **Add transform** (Adicionar transformação).

Quais transformações você acha que mais afetam o treinamento do modelo?

Para obter etapas detalhadas que mostram como preparar dados usando o SageMaker Data Wrangler, consulte <a href="#task1-4-1" target="_self">**Preparar dados usando o SageMaker Data Wrangler (Tarefa 1.4)**</a> na seção *Apêndice*.

Se você quiser importar um conjunto de exemplo de dados processados, consulte <a href="#task1-4-2" target="_self">**Importar um conjunto de exemplo de dados processados (Tarefa 1.4)**</a> na seção *Apêndice*. 


## Desafio 2: criar grupos de recursos no SageMaker Feature Store

Agora que você processou o conjunto de dados, crie recursos e grupos de recursos que serão usados em análises futuras. Use o SageMaker Feature Store para armazenar esses itens em um grupo de recursos e consultá-los durante o treinamento do modelo.

Para realizar essa tarefa, conclua todas estas subtarefas:

1. Exportar recursos para o SageMaker Feature Store.
2. Consultar o grupo de recursos em um armazenamento off-line com o Amazon Athena.

Este desafio exige cerca de *30* minutos para ser concluído.

### Tarefa 2.1: Exportar recursos para o SageMaker Feature Store

<a id="task2-1-continue"></a>

Use o recurso **Export to** (Exportar para) do SageMaker Data Wrangler para criar um caderno Jupyter personalizado. O notebook cria uma definição de recurso e um grupo de recursos. O notebook ingere os registros no grupo de recursos. No notebook, faça as seguintes etapas:

- Defina os valores de registro e de event_time.
- Execute as células do notebook para criar o grupo de recursos.
- Execute as células do notebook para confirmar o grupo de recursos criado.
- Execute as células do notebook para fazer a ingestão dos registros no grupo de recursos.

**Dica 1**: todas essas etapas podem ser realizadas no SageMaker Studio. Ao concluir a criação do grupo de recursos, retorne a este notebook para continuar a Tarefa 2.2.

**Dica 2**: adicione uma transformação personalizada para criar a coluna **event_time**. 

**Dica 3**: o código para adicionar a transformação personalizada é o seguinte:
    


In [ ]:
import time
import datetime
import pandas as pd
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


**Dica 4**: no final do fluxo do SageMaker Data Wrangler, selecione o ícone **+**, escolha a opção **Export to** (Exportar para) e escolha **SageMaker Feature Store (via JupyterNotebook)** (SageMaker Feature Store (por meio do caderno Jupyter)).

**Dica 5**: é possível desativar o armazenamento on-line alterando o valor de **enable_online_store** de **True** para **False**.

Como você usaria o SageMaker Feature Store para armazenar e consultar registros para treinamento, não para inferência?

Para obter etapas detalhadas que mostram como criar um grupo de recursos usando a opção **Export to** (Exportar para), consulte <a href="#task2-1" target="_self">**Criar um grupo de recursos usando a opção Exportar para (Tarefa 2.1)**</a> na seção *Apêndice*. 

Depois de criar um grupo de recursos e fazer a ingestão de dados nele, você terá concluído esta tarefa.

### Tarefa 2.2: Consultar o grupo de recursos em um armazenamento off-line com o Athena

<a id="task2-2-continue"></a>

Use o Athena para extrair registros de um armazenamento de dados off-line. No próximo desafio, você vai dividir esses registros em conjuntos de treinamento, teste e validação.

Use a célula de código fornecida abaixo para fazer as chamadas de API do Amazon Athena. Você também poderia usar o console do Amazon Athena para fazer a consulta, mas isso não faz parte do escopo deste laboratório.

**Dica 1**: é possível criar uma consulta do Athena com **feature_group.athena_query()** e obter o nome da tabela com **query.table_name**.

**Dica 2**: é possível executar uma consulta com **query.run(query_string=query_string, output_location=output_location)** e ler o valor retornado como um dataframe com **query.as_dataframe()**.

Como você poderia usar **event_time** para acompanhar recursos que ocorrem em diferentes pontos na linha do tempo do conjunto de dados?

Para obter as etapas detalhadas que mostram como extrair registros de um armazenamento de dados off-line com o Athena, consulte <a href="#task2-2" target="_self">**Extrair registros de um armazenamento off-line com o Athena (Tarefa 2.2)**</a> na seção *Apêndice*.

Depois de salvar a consulta do Athena retornada como uma variáveis do dataframe, você terá concluído esta tarefa.

In [ ]:
#add_your_task_2_2_code_here



## Desafio 3: Treinar o modelo

O modelo está pronto para ser treinado. Divida os dados em conjuntos de dados de treinamento, teste e validação e treine o modelo. 

O SageMaker Autopilot já foi executado nesses dados e obteve **F1** igual a **0,616**, **accuracy** igual a **0,978**, **AUC** igual a **0,918** e **Recall** igual a **0,539**. Para saber mais sobre as métricas que o SageMaker Autopilot produz, consulte o documento *autopilot-metrics-validation* na seção *Recursos adicionais* e obtenha mais informações.

Durante o treinamento e o ajuste, trabalhe para atingir ou exceder as pontuações do SageMaker Autopilot e confirme se o Amazon SageMaker Debugger não está relatando erros.

Para concluir essa tarefa, realize estas subtarefas:

- Criar um experimento e uma execução.
- Dividir os dados em conjuntos de dados de treinamento, teste e validação.
- Configurar e executar um trabalho de treinamento.
    - Executar um trabalho de treinamento básico.
    - Executar um trabalho de treinamento com o SageMaker Debugger habilitado e analisar os relatórios (opcional).
- Realizar ajuste de hiperparâmetros.

Este desafio exige cerca de *110* minutos para ser concluído.


### Tarefa 3.1: Nomear um experimento e uma execução

<a id="task3-1-continue"></a>

Defina variáveis para nomear o experimento e as execuções. 
Um experimento exige **experiment_name**, **run_name** e **description**. 

Para obter etapas detalhadas de como criar as variáveis, consulte <a href="#task3-1" target="_self">**Nomear um experimento e uma execução (Tarefa 3.1)**</a> na seção *Apêndice*.


In [ ]:
#add_your_task_3_1_code_here



### Tarefa 3.2: Dividir os dados em conjuntos de dados de treinamento, teste e validação

<a id="task3-2-continue"></a>

Use os recursos consultados do SageMaker Feature Store e divida os dados em conjuntos de dados de treinamento, teste e validação.

**Dica 1**: use **np.split** para dividir os conjuntos de dados em três partições.

**Dica 2**: use **to_csv** para criar os arquivos CSV e use **S3Uploader.upload** para adicioná-los ao Amazon S3.

**Dica 3**: o produto final da divisão deve ser uma variável **data_inputs** com valores para **train** e **validation**.

Para obter as etapas detalhadas que mostram como dividir os dados em conjuntos de dados de treinamento, teste e validação, consulte <a href="#task3-2" target="_self">**Dividir dados em conjuntos de dados de treinamento, teste e validação (Tarefa 3.2)**</a> na seção *Apêndice*. 

Depois de dividir os dados em conjuntos de dados de treinamento, teste e validação, você terá concluído esta tarefa.

In [ ]:
#add_your_task_3_2_code_here



### Tarefa 3.3: Configurar e executar um trabalho de treinamento

<a id="task3-3-continue"></a>

Comece seu primeiro trabalho de treinamento configurando o contêiner, um estimador e os hiperparâmetros. Depois, treine o modelo com **fit()**. Se você quiser examinar relatórios mais detalhados, habilite o SageMaker Debugger com **DebuggerHookConfig**.

**Dica 1**: use o contêiner **XGBoost** com a versão **1.5-1**.

**Dica 2**: para começar, defina os hiperparâmetros para **eta**, **gamma**, **max_depth**, **min_child_weight**, **num_round**, **objective** e **subsample**.

**Dica 3**: use a variável **data_inputs** que você já criou no Desafio 1 como o valor de **inputs** do trabalho de treinamento.

**Dica 4**: é possível configurar um trabalho de treinamento definindo as entradas e experiment_config. O experiment_config deve conter uma **sagemaker_session**, um **run_name** e um **experiment_name**.

**Dica 5**: se você quiser usar o SageMaker Debugger, configure **DebuggerHookConfig**, **ProfilerConfig** e o objeto **rule** do Debugger.

Quais hiperparâmetros têm a maior probabilidade de causar um grande impacto no desempenho e na precisão do modelo? Quais hiperparâmetros você planeja ajustar primeiro?

Para obter etapas detalhadas que mostram como configurar e executar um trabalho de treinamento básico, veja <a href="#task3-3-1" target="_self">**Configurar e executar um trabalho de treinamento básico (Tarefa 3.3)**</a> na seção *Apêndice*.

Para obter etapas detalhadas que mostram como configurar e executar um trabalho de treinamento com o Debugger habilitado e analisar relatórios, veja <a href="#task3-3-2" target="_self">**Configurar e executar um trabalho de treinamento com o SageMaker Debugger habilitado e analisar relatórios (Tarefa 3.3)**</a> na seção *Apêndice*.

Depois de terminar um ou mais trabalhos de treinamento, você terá concluído esta tarefa.

In [ ]:
#add_your_task_3_3_code_here



### Tarefa 3.4: Ajustar os hiperparâmetros

<a id="task3-4-continue"></a>

Agora que você concluiu um trabalho de treinamento e fez a análise, ajuste os intervalos de hiperparâmetros considerando o que você já descobriu e execute mais trabalhos de treinamentos para melhorar o modelo.

**Dica 1**: para começar, defina os intervalos de hiperparâmetro para **alpha**, **eta**, **max_depth**, **min_child_weight** e **num_round**.

**Dica 2**: ao executar o **HyperparameterTuner**, defina **objective_metric_name** e **objective_type** considerando o que você já descobriu.

**Dica 3**: para verificar se houve melhora nos resultados do SageMaker Autopilot, abra o menu **Experiments and runs** (Experimentos e execuções) em **SageMaker resources** (Recursos do SageMaker). Na execução, veja as **Métricas**. **ObjectiveMetric** deve ser superior à pontuação de **F1**, que é igual a **0,616**, e **validation:auc** deve ter um **Valor final** superior à pontuação do SageMaker Autopilot, que é igual a **0,918**.

Quando você ajustou os hiperparâmetros, qual deles melhorou mais o desempenho do modelo?

Para obter etapas detalhadas que mostram como configurar intervalos de hiperparâmetros, veja <a href="#task3-4" target="_self">**Configurar intervalos de hiperparâmetros (Tarefa 3.4)**</a> na seção *Apêndice*.

Depois de configurar os intervalos de hiperparâmetros de treinamento e iniciar mais trabalhos de treinamento, você terá concluído esta tarefa.

In [ ]:
#add_your_task_3_4_code_here



## Desafio 4: Avaliar o viés do modelo

Agora que o modelo está treinado, avalie-o usando o Amazon SageMaker Clarify. Se você encontrar problemas, remova o desequilíbrio detectado e retreine o modelo.

Para concluir essa tarefa, realize estas subtarefas:

- Criar um modelo usando o trabalho de treinamento.
- Criar uma configuração de modelo do SageMaker Clarify.
- Criar uma configuração de viés do SageMaker Clarify.
- Usar um trabalho de processador do SageMaker Clarify para gerar relatórios de viés, dados e modelo.
- Remover o desequilíbrio detectado com o SageMaker Clarify (opcional).
- Retreinar o modelo (opcional).

Este desafio exige cerca de *80* minutos para ser concluído.

### Tarefa 4.1: Criar um modelo usando o trabalho de treinamento

<a id="task4-1-continue"></a>

Crie um modelo do XGBoost chamando **create_model** com **model_name**, **role** e **container_def** definidos por você.

**Dica 1**: chame **xgb.create_model()** e escolha um nome para o modelo.

**Dica 2**: use sua sessão e chame **create_model**, passando **model_name**, **role** e **container_def**.

Para obter etapas detalhadas que mostram como criar um modelo, veja <a href="#task4-1" target="_self">**Criar um modelo (Tarefa 4.1)**</a> na seção *Apêndice*.

Depois de criar um modelo, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_1_code_here



### Tarefa 4.2: Criar uma configuração de modelo do SageMaker Clarify

<a id="task4-2-continue"></a>

Crie uma configuração de modelo do SageMaker Clarify usando **SageMakerClarifyProcessor**.

**Dica 1**: defina **instance_count** e **instance_type**.

**Dica 2**: use **role** e **session** criados no início do laboratório de projeto final.

Para obter etapas detalhadas que mostram como criar uma configuração de modelo do SageMaker Clarify, consulte <a href="#task4-2" target="_self">**Criar uma configuração de modelo do SageMaker Clarify (Tarefa 4.2)**</a> na seção *Apêndice*.

Depois de criar uma configuração de modelo do SageMaker Clarify, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_2_code_here



### Tarefa 4.3: Criar uma configuração de viés do SageMaker Clarify

<a id="task4-3-continue"></a>

Crie uma configuração de dados, uma configuração de modelo, uma configuração de rótulo e uma configuração de viés.

**Dica 1**: comece com uma **DataConfig**, configurando o caminho de entrada, o caminho de saída, os cabeçalhos e o tipo de conjunto de dados.

**Dica 2**: depois, crie um **ModelConfig**, escolhendo o conteúdo e o tipo de aceitação, o nome do modelo, o tipo de instância e a contagem de instâncias.

**Dica 3**: em seguida, crie uma configuração **ModelPredictedLabelConfig**, definindo o limite de probabilidade.

**Dica 4**: por último, crie uma **BiasConfig**, definindo os valores ou o limite de rótulo, o nome da faceta e os valores ou o limite de faceta.

Quais facetas você quer explorar primeiro no relatório de viés? Há recursos que são mais suscetíveis a viés?

Para obter etapas detalhadas que mostram como criar uma configuração de viés do SageMaker Clarify, consulte <a href="#task4-3" target="_self">**Criar uma configuração de viés do SageMaker Clarify (Tarefa 4.3)**</a> na seção *Apêndice*.

Depois de criar uma configuração de viés do SageMaker Clarify, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_3_code_here



### Tarefa 4.4: Usar um trabalho de processador do SageMaker Clarify para gerar relatórios de viés, dados e modelo

<a id="task4-4-continue"></a>

Você já pode escolher todas as configurações para os relatórios de viés, dados e modelo. Agora, gere os relatórios.

**Dica 1**: passe os valores de **data_config**, **bias_config**, **model_predicted_label_config** e **model_config** para **run_bias**.

**Dica 2**: é necessário definir **pre_training_methods** e **post_training_methods**.

Para obter etapas detalhadas que mostram como gerar relatórios de viés, dados e modelo usando o SageMaker Clarify, consulte <a href="#task4-4" target="_self">**Gerar relatórios de viés, dados e modelo usando o SageMaker Clarify (Tarefa 4.4)**</a> na seção *Apêndice*.

Depois de usar um trabalho de processador do SageMaker Clarify para gerar relatórios de viés, dados e modelo, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_4_code_here



### Tarefa 4.5: Remover o desequilíbrio detectado com o SageMaker Clarify (opcional)

<a id="task4-5-continue"></a>

Há várias maneiras de remover o desequilíbrio detectado com o SageMaker Clarify. Use qualquer método que você já conheça. Neste laboratório, é demonstrado um exemplo de técnica de sobreamostragem minoritária sintética (SMOTE) que remove o viés de uma das colunas.

**Dica 1**: quando você concluir a remoção do desequilíbrio e quiser testar novamente, crie um dataframe de reamostragem. Na próxima tarefa, você vai criar e carregar um arquivo CSV.

Para obter etapas detalhadas que mostram como remover o desequilíbrio, consulte <a href="#task4-5" target="_self">**Remover o desequilíbrio (Tarefa 4.5)**</a> na seção *Apêndice*.

Depois de remover o desequilíbrio detectado com o SageMaker Clarify, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_5_code_here



### Tarefa 4.6: Retreinar o modelo (opcional)

<a id="task4-6-continue"></a>

Carregue o novo arquivo no Amazon S3. Depois, crie um estimador e retreine com os novos dados.

**Dica 1**: use **s3_client.upload_file** para carregar o novo arquivo no bucket.

**Dica 2**: use **xgboost_starter_script.py** e chame **XGBoost**. Depois, retreine o modelo com os novos dados.

O modelo retreinado atingiu uma pontuação de F1 superior? Se você usou o SageMaker Debugger, conseguiu resolver todos os problemas descobertos?

Para obter etapas detalhadas que mostram como retreinar o modelo, consulte <a href="#task4-6" target="_self">**Retreinar o modelo (Tarefa 4.6)**</a> na seção *Apêndice*.

Depois de retreinar o modelo, você terá concluído esta tarefa.

In [ ]:
#add_your_task_4_6_code_here



## Desafio 5: Transformação em lote

O modelo está pronto para implantação. Use um trabalho de transformação em lote com registros em lote e veja os dados de previsão e precisão no Amazon S3. Depois, limpe algumas das instâncias do SageMaker.

Para concluir essa tarefa, realize estas subtarefas:

- Criar um trabalho de transformação em lote para o modelo.
- Visualizar os dados de previsão no Amazon S3.
- Limpar instâncias do SageMaker (opcional).

Este desafio exige cerca de *40* minutos para ser concluído.

### Tarefa 5.1: Criar um trabalho de transformação em lote para o modelo

<a id="task5-1-continue"></a>

Crie um trabalho de transformação em lote usando **transformer** no estimador de modelo. Depois, execute o trabalho em lote.

**Dica 1**: use o transformador e configure **instance_count**, **instance_type**, **strategy**, **assemble_with** e **output_path**.

**Dica 2**: envie os dados de teste, listados em **test_path**, ao endpoint e aguarde os resultados.

Para obter etapas detalhadas que mostram como criar um trabalho de transformação em lote, consulte <a href="#task5-1" target="_self">**Criar um trabalho de transformação de lote (Tarefa 5.1)**</a> na seção *Apêndice*.

Depois de criar um trabalho de transformação em lote e executá-lo com um conjunto de registros, você terá concluído esta tarefa.

In [ ]:
#add_your_task_5_1_code_here



### Tarefa 5.2: Visualizar os dados de previsão no Amazon S3

<a id="task5-2-continue"></a>

Quando o trabalho de transformação em lote estiver concluído, leia os dados do Amazon S3. 

**Dica 1**: é possível copiar os dados da saída do transformador usando **%aws s3 cp --recursive $transformer.output_path ./**

**Dica 2**: quando você tiver os dados, poderá vê-los com **%head test_data_batch.csv.out**

Observe as previsões. Há previsões surpreendentes?

Para obter etapas detalhadas que mostram como visualizar dados de previsão de um trabalho de transformação em lote, consulte <a href="#task5-2" target="_self">**Visualizar dados de previsão de um trabalho de transformação em lote (Tarefa 5.2)**</a> na seção *Apêndice*.

Depois de visualizar os dados do trabalho de transformação em lote, você terá concluído esta tarefa.

In [ ]:
#add_your_task_5_2_code_here



### Tarefa 5.3: Limpar instâncias do SageMaker (opcional)

Para manter um nível baixo de custos, a prática recomendada é excluir as instâncias que não estão sendo mais usadas. É possível excluir instâncias rapidamente usando o SageMaker Studio. Agora, dedique um momento para abrir os recursos atuais no SageMaker Studio e fechar as instância restantes.

Se você planeja fazer a próxima tarefa de pipeline, **deixe a instância do notebook em execução**.

**Dica 1**: é possível visualizar uma lista de instâncias em execução clicando no ícone **Running Terminals and Kernels** (Terminais e kernels em execução) no SageMaker Studio.

**Dica 2**: é possível usar o ícone **Shut down** (Fechar) para interromper a instância.

<a id="task5-3-continue"></a>

Para obter etapas detalhadas que mostram como limpar instâncias do SageMaker no SageMaker Studio, consulte <a href="#task5-3" target="_self">**Limpar instâncias do SageMaker no SageMaker Studio (Tarefa 5.3)**</a> na seção *Apêndice*.

Depois de parar todas as instâncias do SageMaker no SageMaker Studio, você terá concluído esta tarefa.

### Desafio 6: Criar um pipeline automatizado (opcional)

Agora que você já usou o SDK Python do Amazon SageMaker e o Amazon SageMaker Studio para um fluxo de trabalho de machine learning (ML), use o SageMaker Pipelines para dimensionar o fluxo de trabalho. Continue no script do pipeline fornecido no ambiente de laboratório para realizar este desafio. 

- Crie as etapas do pipeline.
    - Consulte os dados processados do SageMaker Feature Store.
    - Treine e ajuste o modelo.
    - Avalie o modelo treinado.
    - Realize um trabalhos de transformação em lote.
    - Registre um modelo.
    - Avalie o treinamento do modelo com o SageMaker Clarify.
- Defina e inicie o pipeline.
- Visualize o ML Lineage Tracking.

Este desafio exige cerca de *120* minutos para ser concluído.


### Tarefa 6.1: Configurar um pipeline

<a id="task6-1-continue"></a>

Use um modelo de pipeline e configure entradas e saídas. Quando as configurações estiverem prontas, execute o pipeline. O pipeline pode incluir uma ampla variedade de etapas. Veja uma lista de sugestões de etapas de configuração:
- **AutoModelProcess**: uma etapa **Processing** que obtém o arquivo .csv e divide-o em conjuntos de dados de treinamento, teste e validação.
- **AutoHyperParameterTuning**: uma etapa **Tuning** que obtém um intervalo de hiperparâmetros e ajusta o modelo.
- **AutoEvalBestModel**: uma etapa **Processing** que cria um relatório de avaliação para descrever o melhor modelo.
- **CheckAUCScoreAutoEvaluation**: uma etapa **Condition** que avalia os modelos com base em uma métrica de avaliação. 
- **AutoCreateModel**: uma etapa **Model** que cria um modelo.
- **RegisterAutoModel-RegisterModel**: uma etapa **RegisterModel** que registra um modelo.
- **AutoModelConfigFile**: uma etapa **Processing** que cria um relatório de viés.
- **AutoTransform**: uma etapa **Transform** que executa um trabalho de transformação em lote.
- **ClarifyProcessingStep**: uma etapa **Processing** que executa um trabalho do SageMaker Clarify.

**Dica 1**: há várias etapas de pipeline que você pode escolher. Para saber mais sobre as etapas de pipeline e visualizar o código de exemplo de cada etapa, consulte o documento *Criar e gerenciar etapas de pipeline* na seção *Recursos adicionais*.

**Dica 2**: inicie com uma etapa **Processing** para obter os dados. Depois, crie uma etapa **Tuning** para ajustar o modelo. Depois, crie uma etapa **Model** para criar o modelo.

**Dica 3**: as etapas detalhadas contêm uma solução de exemplo e incluem etapas para criar relatórios de avaliação, gerar um relatório de viés, executar um trabalho de transformação em lote e executar um trabalho do SageMaker Clarify. 

Para obter etapas detalhadas que mostram como configurar um pipeline, veja <a href="#task6-1" target="_self">**Configurar um pipeline (Tarefa 6.1)**</a> na seção *Apêndice*.

Depois de configurar o pipeline e iniciar o trabalho de pipeline, você terá concluído esta tarefa.

In [ ]:
#add_your_task_6_1_code_here



### Tarefa 6.2: Monitorar o pipeline

<a id="task6-2-continue"></a>

Monitore o pipeline durante a execução, visualizando as entradas e as saídas. 

**Dica 1**: use “RunPipeline.describe()” para descrever o pipeline recém-criado.

**Dica 2**: é possível visualizar as etapas do pipeline em execução na IU do SageMaker Studio. Abra o menu **SageMaker resources** (Recursos do SageMaker), selecione **Pipelines** e escolha o pipeline que você criou. 

Para obter etapas detalhadas que mostram como monitorar um pipeline, consulte <a href="#task6-2" target="_self">**Monitorar um pipeline**</a> no *Apêndice*.

Ao terminar de monitorar o pipeline, você terá concluído esta tarefa.

In [ ]:
#add_your_task_6_2_code_here



Parabéns! Você usou um conjunto de dados de seguro de automóvel para detectar sinistros com possibilidade de fraude. Você explorou uma solução técnica para prever a probabilidade de que um determinado sinistro de seguro de automóvel seja fraudulenta usando o SageMaker Studio e o SDK Python do Amazon SageMaker.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.

## Recursos adicionais

- [métricas do Autopilot](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)
- [Etapa de processamento](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

## Apêndice

<a name="task1-1" id="task1-1"></a>

### Apêndice: Examinar os dados (Tarefa 1.1)

Para examinar os dados, especifique o caminho e carregue os dados usando o Pandas. Dedique um momento para examinar um exemplo das duas tabelas.

In [ ]:
#read-csv-files
claims_data = pd.read_csv("./data/claims_preprocessed.csv", index_col=0)
customers_data = pd.read_csv("./data/customers_preprocessed.csv", index_col=0)

In [ ]:
#claims-data-sample
claims_data.head()

In [ ]:
#customers-data-sample
customers_data.head()

Para continuar este laboratório, retorne à <a href="#task1-1-continue" target="_self">Tarefa 1.1</a>.

<a name="task1-2-1" id="task1-2-1"></a>

### Apêndice: Explorar um conjunto de dados no SageMaker Studio (Tarefa 1.2)

Comece a exploração de dados no SageMaker Data Wrangler. Importe os arquivos do bucket do S3 e analise os dados.

A próxima etapa abrirá uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **capstone.ipynb** para a lateral ou selecione a aba **capstone.ipynb** e, na barra de ferramentas, escolha **File** (Arquivo) e **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o grupo de recursos.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções.

1. Na lateral esquerda do SageMaker Studio, escolha o ícone **Home** (Página inicial).
1. Expanda a seção **Data** (Dados) e escolha **Data Wrangler**.

O SageMaker Studio abre a aba **Data Wrangler**.

1. Escolha **+** **Create Data Wrangler flow** (+ Criar fluxo do Data Wrangler).

O SageMaker Studio abre a aba **untitled.flow**.

1. Aguarde a conclusão do carregamento da aba **untitled.flow**, indicada por uma barra de progresso. A conclusão pode demorar de dois a três minutos.

O SageMaker Studio abre a página **Create connection** (Criar conexão) da aba *Data Wrangler*.

1. Abra o menu de contexto (clique com o botão direito do mouse) na aba do arquivo **untitled.flow** e, para alterar o nome do arquivo, escolha **Rename Data Wrangler Flow...** (Renomear o fluxo do Data Wrangler...).

O SageMaker Studio abre a janela de mensagem **Rename File** (Renomear arquivo).

1. Em **New Name** (Novo nome), insira “CapstoneDataWrangler.flow”.
1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Rename (Renomear)</span>.

A janela de mensagem **Rename File** (Renomear arquivo) é fechada.

1. Na aba *CapstoneDataWrangler.flow*, na seção **Fontes de dados**, escolha **Amazon S3**.

O SageMaker Studio abre a página **Import a dataset from S3** (Importar um conjunto de dados do S3) dentro da aba *DataWrangler.flow*.

1. Na lista de buckets, abra o bucket que contém **databucket** no nome.
1. Escolha o primeiro conjunto de dados, um arquivo chamado **claims.csv**.

1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Importar</span>.

1. Retorne à visualização **Data flow** (Fluxo de dados), escolha o **&lt; Fluxo de dados** localizado na parte superior esquerda da aba *CapstoneDataWrangler.flow*.
1. Selecione a aba **Import** (Importar), localizada na parte superior esquerda da aba *CapstoneDataWrangler.flow* tab.

O SageMaker Studio abre a página *Create connection** (Criar conexão).

1. Na aba *CapstoneDataWrangler.flow*, na seção **Fontes de dados**, escolha **Amazon S3**.

O SageMaker Studio abre a página **Import a dataset from S3** (Importar um conjunto de dados do S3) dentro da aba *DataWrangler.flow*.

1. Na lista de buckets, abra o bucket que contém **databucket** no nome.
1. Escolha o segundo conjunto de dados, um arquivo chamado **customers.csv**.
1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Importar</span>.

1. Retorne à visualização **Data flow** (Fluxo de dados), escolha o **&lt; Fluxo de dados** localizado na parte superior esquerda da aba *CapstoneDataWrangler.flow*.

1. Na aba Fluxo de dados, selecione o sinal **+** ao lado do ícone **Data types** (Tipos de dados) e escolha **Get data insights** (Obter informações de dados).

1. Crie o relatório e explore as informações.

1. Retorne à aba **Data flow** (Fluxo de dados), selecione o sinal **+** ao lado do ícone **Data types** (Tipos de dados) e escolha **Add analysis** (Adicionar análise).

1. Crie a análise e explore os resultados.

Use qualquer tipo de relatório que ajude a explorar os conjuntos de dados por completo. Ao concluir, continue na próxima tarefa.

Para continuar este laboratório, retorne à <a href="#task1-2-continue" target="_self">Tarefa 1.2</a>.


<a name="task1-2-2" id="task1-2-2"></a>

### Apêndice: Explorar um conjunto de dados no notebook (Tarefa 1.2)

Há muitas maneiras de explorar os conjuntos de dados. Veja vários exemplos de etapas de exploração de dados que você pode seguir. Use-as como referência para iniciar a exploração dos aspectos do conjunto de dados.

In [ ]:
#gender-graph
import matplotlib.pyplot as plt
customers_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

In [ ]:
#fraud-graph
claims_data.fraud.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Not Fraud", "Fraud"]);

In [ ]:
#education-category-graphs
educ = customers_data.customer_education.value_counts(normalize=True, sort=False)
plt.bar(educ.index, educ.values)
plt.xlabel("Customer Education Level");

In [ ]:
#claim-amount-graph
plt.hist(claims_data.total_claim_amount, bins=30)
plt.xlabel("Total Claim Amount")

In [ ]:
#claims-filed-graph
customers_data.num_claims_past_year.hist(density=True)
plt.suptitle("Number of Claims in the Past Year")
plt.xlabel("Number of claims per year")

In [ ]:
#paid-plot-graphs
sns.pairplot(
    data=customers_data, vars=["num_insurers_past_5_years", "months_as_customer", "customer_age"]
);

In [ ]:
#fraud-insurers-graph
combined_data = customers_data.join(claims_data)
sns.lineplot(x="num_insurers_past_5_years", y="fraud", data=combined_data);

In [ ]:
#months-as-customer-graph
sns.boxplot(x=customers_data["months_as_customer"]);

In [ ]:
#customer-age-graph
sns.boxplot(x=customers_data["customer_age"]);

In [ ]:
#fraud-gender-graph
combined_data.groupby("customer_gender_female").mean()["fraud"].plot.bar()
plt.xticks([0, 1], ["Male", "Female"])
plt.suptitle("Fraud by Gender");

In [ ]:
#correlation-matrix-graph
cols = [
    "fraud",
    "customer_gender_male",
    "customer_gender_female",
    "months_as_customer",
    "num_insurers_past_5_years",
]
corr = combined_data[cols].corr()

# plot the correlation matrix
sns.heatmap(corr, annot=True, cmap="Reds");

In [ ]:
#load-combined-data
combined_data = pd.read_csv("./data/claims_customer.csv")

In [ ]:
#remove-unnecessary-columns
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains("^Unnamed: 0")]
combined_data.head()

In [ ]:
#describe-combined-data
combined_data.describe()

In [ ]:
#generate-statistics
combined_stats = []

for col in combined_data.columns:
    combined_stats.append(
        (
            col,
            combined_data[col].nunique(),
            combined_data[col].isnull().sum() * 100 / combined_data.shape[0],
            combined_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            combined_data[col].dtype,
        )
    )

stats_df = pd.DataFrame(
    combined_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)
stats_df.sort_values("percent_largest_category", ascending=False)

In [ ]:
#heatmap-graph
sns.set_style("white")

corr_list = [
    "customer_age",
    "months_as_customer",
    "total_claim_amount",
    "injury_claim",
    "vehicle_claim",
    "incident_severity",
    "fraud",
]

corr_df = combined_data[corr_list]
corr = round(corr_df.corr(), 2)

fix, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, mask=mask, ax=ax, annot=True, cmap="OrRd")

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10, ha="right", rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10, va="center", rotation=0)

plt.show()

Para continuar este laboratório, retorne à <a href="#task1-2-continue" target="_self">Tarefa 1.2</a>.

<a name="task1-3-1" id="task1-3-1"></a>

### Apêndice: Unificar tabelas no SageMaker Studio (Tarefa 1.3)

1. Retorne à visualização **Data flow** (Fluxo de dados), escolha o **&lt; Fluxo de dados** localizado na parte superior esquerda da aba *CapstoneDataWrangler.flow*.
1. Selecione o sinal **+** ao lado do ícone **claims.CSV Data types** (Tipos de dados claims.CSV) e, no menu de contexto, escolha **Join** (Unificar).

O SageMaker Data Wrangler exibe a página **Unificar**.

1. Escolha o ícone **customers.csv Data types** (Tipos de dados customers.csv).
1. Selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configurar</span>.
1. Em **Join Type** (Tipo de junção), selecione **Inner** (Interna).
1. Na seção **Columns** (Colunas):

    - Em **Left** (Esquerda), selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.
    
    - Em **Right** (Direita), selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.

1. Escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Visualizar</span>.

1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Adicionar</span>.

Para continuar este laboratório, retorne à <a href="#task1-3-continue" target="_self">Tarefa 1.3</a>.

<a name="task1-3-2" id="task1-3-2"></a>

### Apêndice: Gerar um relatório de viés de pré-treinamento (Tarefa 1.3)

Crie um relatório de viés do SageMaker Clarify usando um fluxo do SageMaker Data Wrangler.

1. Escolha a aba **CapstoneDataWrangler.flow**.
1. Acesse a visualização **Data flow** (Fluxo de dados). Se necessário, escolha o **&lt; Data flow** (&lt; Fluxo de dados) localizado na parte superior esquerda da aba *DataWranglerLab.flow*. 
1. Selecione o sinal **+** ao lado do ícone **Join** (Unificar) e, no menu de contexto, escolha **Add analysis** (Adicionar análise).
1. Na seção **Create analysis** (Criar análise):

- Em **Analysis type** (Tipo de análise), selecione **Bias Report** (Relatório de viés).
- Em **Analysis name** (Nome da análise), insira “viés de fraude por idade”.
- Em **Select the column your model predicts (target)** (Selecionar a coluna que o modelo prevê (destino)), selecione **fraud**.
- Em **Is your predicted column a value or threshold?** (A coluna prevista é um valor ou um limite?), escolha a opção **value** (valor).
- Em **Valores previstos**, insira **1**.
- Em **Select the column to analyze for bias** (Selecionar a coluna para analisar o viés), selecione **customer_age**.

1. Selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Check for bias (Verificar viés)</span>.

Após a conclusão do trabalho, visualize as métricas retornadas. Observe se há viés e planeje as etapas de processamento que deverão ocorrer na coluna analisada. 

1. Selecione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Salvar</span> para manter as análises que deseja salvar.

Você pode repetir essas etapas em todas as colunas em que deseja analisar o viés.

Para continuar este laboratório, retorne à <a href="#task1-3-continue" target="_self">Tarefa 1.3</a>.

<a name="task1-4-1" id="task1-4-1"></a>

### Apêndice: Preparar dados usando o SageMaker Data Wrangler (Tarefa 1.4)

Combine os conjuntos de dados. Unifique-os no SageMaker Data Wrangler usando o **policy_id**.

Com o SageMaker Data Wrangler, é possível unificar dados em qualquer ponto no fluxo. É possível concluir a preparação de dados em arquivos individuais antes de unificá-los ou transformar os recursos após a unificação. Um fluxo do SageMaker Data Wrangler é flexível.

Se você não unificou as tabelas na tarefa 1.3, as etapas a seguir mostrarão esse processo.

1. Retorne à visualização **Data flow** (Fluxo de dados), escolha o **&lt; Fluxo de dados** localizado na parte superior esquerda da aba *CapstoneDataWrangler.flow*.
1. Selecione o sinal **+** ao lado do ícone **Data types** (Tipos de dados) e, no menu de contexto, escolha **Join** (Unificar).

O SageMaker Data Wrangler exibe a página **Join** (Unificar).

1. Selecione o segundo ícone **Data types** (Tipos de dados).
1. Selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configurar</span>.
1. Em **Join Type** (Tipo de junção), selecione **Inner** (Interna).
1. Na seção **Columns** (Colunas):

    - Em **Left** (Esquerda), selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.
    
    - Em **Right** (Direita), selecione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.

1. Escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Visualizar</span>.

1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Adicionar</span>.

Com as tabelas de dados unificadas, transforme os dados combinados.

1. Retorne à visualização **Data flow** (Fluxo de dados), escolha o **&lt; Fluxo de dados** localizado na parte superior esquerda da aba *CapstoneDataWrangler.flow*.
1. Selecione o sinal **+** ao lado do ícone **Join** (Unificar) e, no menu de contexto, escolha **Add transform** (Adicionar transformação).

É possível adicionar várias transformações usando esse menu. Uma visualização do conjunto de dados é mostrada do lado esquerdo do menu de transformação.

Adicione as seguintes etapas de transformação ao fluxo do SageMaker Data Wrangler:
- Codificar de maneira categórica (codificação one-hot): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type** e **policy_state** usando uma estratégia de manipulação de **Skip** (ignorar) inválidos, selecione o estilo de saída **Columns** (Colunas).
- Codificar de maneira categórica (codificação ordinal): **customer_education**, **incident_severity**, **police_report_available** e **policy_liability** usando uma estratégia de manipulação de **Skip** (ignorar) inválidos.
- Analisar coluna como tipo: **vehicle_claim** e **total_claim_amount** de **Float** para **Long**.
- Gerenciar colunas (Remover coluna): **customer_zip** e **policy_id_1**.
- Gerenciar colunas (Mover coluna): **fraud** (usando **Mover para início**).
- Gerenciar colunas (Renomear colunas): substitua o símbolo **/** de **collision_type_N/A** e de **driver_relationship_N/A** por um **_**.
- Gerenciar colunas (Renomear coluna): renomeie **policy_id_0** como **policy_id**.

Se algum nome de coluna contiver um caractere **/**, renomeie a coluna para substituir **/** por **_**. Se algum nome de coluna contiver um caractere de espaço em branco, renomeie a coluna para substituir o espaço em branco por **_**. Por exemplo, toda coluna criada com a codificação one-hot que tiver **N/A** como valor, precisará ser renomeada. O SageMaker Feature Store não aceita colunas que contenham o caractere **/** ou espaço em branco.

Quando você tiver transformado os dados e puder começar a treinar o modelo, continue na próxima tarefa. É possível voltar a este fluxo a qualquer momento para fazer alterações considerando as descobertas feitas durante o treinamento e o ajuste.


Para continuar este laboratório, retorne à <a href="#task1-4-continue" target="_self">Tarefa 1.4</a>.

<a name="task1-4-2" id="task1-4-2"></a>

### Apêndice: Importar um conjunto de exemplo de dados processados (Tarefa 1.4)

Se o pré-processamento for interrompido ou se você quiser carregar um conjunto de dados que já foi processado, acesse os dados processados armazenados na pasta de dados no bucket do S3.

In [ ]:
#processed-data-import
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims_customer.csv")
df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
df_processed.head()

Para continuar este laboratório, retorne à <a href="#task1-4-continue" target="_self">Tarefa 1.4</a>.

<a name="task2-1" id="task2-1"></a>

### Apêndice: Criar um grupo de recursos usando a opção Exportar para (Tarefa 2.1)

O SageMaker Data Wrangler pode exportar dados para o SageMaker Feature Store. Ele cria um notebook com todo o código necessário para configurar um grupo de recursos e fazer a ingestão dos dados transformados no grupo de recursos.

1. Escolha **Add step** (Adicionar etapa).

1. Escolha **Custom transform** (Transformação personalizada).

1. Em **Name** (Nome), insira “event_time”. 

1. Selecione **Python (PySpark)**, se essa opção ainda não estiver selecionada.

1. Em **Your custom transform** (Sua transformação personalizada), insira o seguinte:


In [ ]:
import time
import datetime
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


1. Escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Visualizar</span>.

1. Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Adicionar</span>.

Essa opção adiciona **event_time** como uma coluna ao conjunto de dados. O SageMaker Feature Store exige um **event_time** e um ID de **record** exclusivos. Use **policy_id** como o ID de **record**.

1. Para retornar ao fluxo de dados, selecione o ícone **&lt; Data flow** (&lt; Fluxo de dados).

1. Selecione o ícone **+** ao lado das suas transformações no SageMaker Data Wrangler.

1. Selecione **Export to** (Exportar dados).

1. Escolha **SageMaker Feature Store (via Jupyter Notebook)** (SageMaker Feature Store (por meio do caderno Jupyter)).

Um novo notebook é aberto.

1. Na primeira célula, altere as seguinte variáveis:
- Em **record_identifier_feature_name**, substitua **None** (Nenhum) por `"policy_id"`. Se você unificou as tabelas de clientes e de sinistros e não removeu a segunda coluna **policy_id**, poderá ser necessário substituir **None** (Nenhum) por `"policy_id_0"`. Não altere o valor de **None** (Nenhum) após a instrução **if**.
- Em **event_time_feature_name**, substitua **None** (Nenhum) por `"event_time"`. Não altere o valor de **None** (Nenhum) após a instrução **if**.

**Saída selecionada:** quando você concluir a edição da célula, ela deverá estar semelhante a este exemplo:


In [ ]:
record_identifier_feature_name = "policy_id"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "event_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


2. Percorra todas as células para criar uma definição de recurso, criar um grupo de recursos e fazer a ingestão dos dados transformados no grupo de recursos usando um trabalho de processamento. 

Quando as células estiverem concluídas, o Feature Store estará pronto para uso.

Para continuar este laboratório, retorne à <a href="#task2-1-continue" target="_self">Tarefa 2.1</a>.

<a name="task2-2" id="task2-2"></a>

### Apêndice: Extrair registros de um armazenamento off-line com o Athena (Tarefa 2.2)

Configure uma consulta do Athena com **athena_query**. Em seguida, defina **query_string**. Por fim, execute a consulta e visualize um exemplo dos resultados.

In [ ]:
#configure-and-run-athena-query
try:
    # If there is a feature group, get the name
    feature_group_name = sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()['FeatureGroupSummaries'][0]['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

    # Confirm the Athena settings are configured
    try:
        boto3.client('athena').update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'EnforceWorkGroupConfiguration':False
            }
        )
    except Exception:
        pass
    
    # Configure the query
    query = feature_group.athena_query()
    table = query.table_name
    query_string = f'SELECT * FROM "{table}" '
    output_location = f"s3://{sagemaker_session.default_bucket()}/query_results/"
    print(f"Athena query output location: \n{output_location}")

    # Run the query
    query.run(query_string=query_string, output_location=output_location)
    query.wait()
    df_feature_store = query.as_dataframe()
    
    # Wait for data to appear in the feature group
    attempts = 0
    while len(df_feature_store.index) == 0 and attempts < 30:
        print("Waiting for feature group to populate...")
        time.sleep(60)
        # Rerun the query
        query.run(query_string=query_string, output_location=output_location)
        query.wait()
        df_feature_store = query.as_dataframe()
        # Increment the attempts
        attempts += 1
    if len(df_feature_store.index) != 0:
        print("The feature group is populated.")
except IndexError as e:
    # If there is no feature group, thrown an error
    print("No feature groups were found. Please create a feature group.")

Depois de executar o bloco de código anterior, o Amazon Athena consulta os registros salvos no bucket do Amazon S3 cujos nomes começam com *sagemaker*. O objeto de consulta salvo fica em um diretório chamado **query_results**.

Para continuar este laboratório, retorne à <a href="#task2-2-continue" target="_self">Tarefa 2.2</a>.

<a name="task3-1" id="task3-1"></a>

### Apêndice: Nomear um experimento e uma execução (Tarefa 3.1)

Para criar um experimento, use a biblioteca **sagemaker.experiments.run**. Defina o **experiment_name**, um **run_name** e a **description**.

In [ ]:
from time import gmtime, strftime

#create experiment and run-names
create_date = strftime("%m%d%H%M")
capstone_experiment_name="capstone-experiment-{}".format(create_date)
capstone_run_name = "lab-capstone-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-capstone', 'Value': 'lab-capstone-run'}]

# provide a description
description="Using SM Experiments with the Auto dataset."

print(f"Experiment name - {capstone_experiment_name},  run name - {capstone_run_name}")

Para continuar este laboratório, retorne à <a href="#task3-1-continue" target="_self">Tarefa 3.1</a>.

<a name="task3-2" id="task3-2"></a>

### Apêndice: Dividir os dados em conjuntos de dados de treinamento, teste e validação (Tarefa 3.2)

Para dividir os dados, use **np.split** e especifique como deseja fazer isso. Depois, crie arquivos CSV e carregue-os no bucket do S3. Em seguida, defina as entradas de treinamento. Por fim, crie a variável **data_inputs**. Use a variável data_inputs durante todo o desafio para especificar os conjuntos de dados de treinamento e de validação ao treinar o modelo.

In [ ]:
#train-validation-test-split
try:
    # If there is a feature group, use it
    df_feature_store = df_feature_store.iloc[: , :-4]
    df_processed_pre_split = df_feature_store
    print("Using the records from the feature group")
except NameError:
    # If there is no feature group, use the processed dataset
    df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
    df_processed_pre_split = df_processed
    print("Using the processed records from Amazon S3")

# Split the data into train, validation, and test datasets
train_data, validation_data, test_data = np.split(
    df_processed_pre_split.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_processed_pre_split)), int(0.9 * len(df_processed_pre_split))],
)

# Create the CSV files and upload them to your default bucket
train_data.to_csv("train_data.csv", index=False, header=False)
validation_data.to_csv("validation_data.csv", index=False, header=False)
test_data.to_csv("test_data.csv", index=False, header=False)

train_path = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
validation_path = S3Uploader.upload("validation_data.csv", "s3://{}/{}".format(bucket, prefix))
test_path = S3Uploader.upload("test_data.csv", "s3://{}/{}".format(bucket, prefix))

# Set the training inputs
train_input = TrainingInput(train_path, content_type="text/csv")
validation_input = TrainingInput(validation_path, content_type="text/csv")
test_input = TrainingInput(test_path, content_type="text/csv")

data_inputs = {
    "train": train_input,
    "validation": validation_input
}

Para continuar este laboratório, retorne à <a href="#task3-2-continue" target="_self">Tarefa 3.2</a>.

<a name="task3-3-1" id="task3-3-1"></a>

### Apêndice: Configurar e executar um trabalho de treinamento básico (Tarefa 3.3)

Se você quiser começar com um trabalho de treinamento básico, use o contêiner **XGBoost** básico. Depois, configure o estimador, indicando o contêiner e a função que deseja usar. Depois de definir essas configurações, escolha os hiperparâmetros. Você pode usar os valores padrão fornecidos no código a seguir ou editá-los considerando suas descobertas durante a preparação de dados.

Para executar o trabalho de treinamento, chame **fit()**, definindo as entradas como a variável **data_inputs** e definindo as configurações de **run_name** e **experiment_name**.

In [ ]:

from sagemaker import image_uris
#train-model
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8

hyperparameters = {
    "eta":eta,
    "gamma":gamma,
    "max_depth":max_depth,
    "min_child_weight":min_child_weight,
    "num_round":num_round,
    "objective":objective,
    "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role,    
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags
)

with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    )        

Para continuar este laboratório, retorne à <a href="#task3-3-continue" target="_self">Tarefa 3.3</a>.

<a name="task3-3-2" id="task3-3-2"></a>

### Apêndice: Configurar e executar um trabalho de treinamento com o SageMaker Debugger ativado e analisar os relatórios (Tarefa 3.3)

O SageMaker Debugger ajuda a encontrar relatórios adicionais que possam informar o ajuste dos hiperparâmetros rapidamente, economizando tempo ao iniciar a execução de mais trabalhos de treinamento com intervalos de hiperparâmetros. Para habilitar o Debugger, configure **DebuggerHookConfig** e **rules**.

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=300
objective='binary:logistic'
subsample=0.7
        
hyperparameters = {
        "eta":eta,
        "gamma":gamma,
        "max_depth":max_depth,
        "min_child_weight":min_child_weight,
        "num_round":num_round,
        "objective":objective,
        "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags,

    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
        ),
        #Set the Debugger Profiler Configuration
        profiler_config = ProfilerConfig(
            system_monitor_interval_millis=500,
            framework_profile_params=FrameworkProfile()
    ),
        #Configure the Debugger Rule Object
        rules = [
            ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
            Rule.sagemaker(rule_configs.create_xgboost_report()),  
            Rule.sagemaker(rule_configs.overfit()),
            Rule.sagemaker(rule_configs.overtraining()),
            Rule.sagemaker(rule_configs.loss_not_decreasing(),
                rule_parameters={
                    "collection_names": "metrics",
                    "num_steps": str(save_interval * 2),
                }
            )
    ]
)
with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)
# Train the model
xgb.fit(
    inputs = data_inputs
) 

Para continuar este laboratório, retorne à <a href="#task3-3-continue" target="_self">Tarefa 3.3</a>.

<a name="task3-4" id="task3-4"></a>

### Apêndice: Configurar os intervalos de hiperparâmetros de treinamento (Tarefa 3.4)

Agora que você já treinou pelo menos um modelo, aplique o que aprendeu com o processamento dos dados e o SageMaker Debugger para informar quais são os intervalos selecionados para os hiperparâmetros. Edite os seguintes intervalos de hiperparâmetros e execute o trabalho de treinamento.

In [ ]:
#tune-model
# Setup the hyperparameter ranges
hyperparameter_ranges = {
    "alpha": ContinuousParameter(0, 2),
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "min_child_weight": ContinuousParameter(1, 10),
    "num_round": IntegerParameter(100, 1000)
}

# Define the target metric and the objective type (max/min)
objective_metric_name = "validation:auc"
objective_type="Maximize"

# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type="Auto"
)

# Tune the model
tuner.fit(
    inputs = data_inputs
)

Para continuar este laboratório, retorne à <a href="#task3-4-continue" target="_self">Tarefa 3.4</a>.

<a name="task4-1" id="task4-1"></a>

### Apêndice: Criar um modelo (Tarefa 4.1)

Crie um modelo do XGBoost chamando **create_model** com **model_name**, **role** e **container_def** definidos por você.

In [ ]:
#model-configurations
model_name = "capstone-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

Para continuar este laboratório, retorne à <a href="#task4-1-continue" target="_self">Tarefa 4.1</a>.

<a name="task4-2" id="task4-2"></a>

### Apêndice: Criar uma configuração de modelo do SageMaker Clarify (Tarefa 4.2)

Crie uma configuração de modelo do SageMaker Clarify usando **SageMakerClarifyProcessor**. Defina **instance_count** e **instance_type**. Use **role** e **session** criados no início do Projeto final.

In [ ]:
#define-clarify-processor
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sagemaker_session
)

Para continuar este laboratório, retorne à <a href="#task4-2-continue" target="_self">Tarefa 4.2</a>.

<a name="task4-3" id="task4-3"></a>

### Apêndice: Criar uma configuração de viés do SageMaker Clarify (Tarefa 4.3)

Para criar uma configuração de viés do SageMaker Clarify, escolha um caminho de saída para os dados e defina o caminho de entrada do trabalho de treinamento, além de **label**, **headers** e **dataset_type**.

Depois, crie **ModelConfig** e **ModelPredictedLabelConfig**.

Por fim, configure **BiasConfig** com os campos que o SageMaker Clarify deverá observar. Você pode adicionar ou remover os campos que deseja explorar, considerando as descobertas iniciais.

In [ ]:
#define-data-config
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
#define-model-config
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
#define-label-config
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
#define-bias-config
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="customer_gender_female", facet_values_or_threshold=[0], group_name="customer_age"
)

Para continuar este laboratório, retorne à <a href="#task4-3-continue" target="_self">Tarefa 4.3</a>.

<a name="task4-4" id="task4-4"></a>

### Apêndice: Gerar relatórios de viés, dados e modelo usando SageMaker Clarify (Tarefa 4.4)

Agora que o trabalho do SageMaker Clarify já está configurado, execute-o chamando **run_bias**.

In [ ]:
#run-bias-report
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

Para continuar este laboratório, retorne à <a href="#task4-4-continue" target="_self">Tarefa 4.4</a>.

<a name="task4-5" id="task4-5"></a>

### Apêndice: Remover desequilíbrio (Tarefa 4.5)

Neste exemplo, você está aumentando a taxa de amostragem de **customer_gender_female** para reduzir o viés no conjunto de dados. Se você encontrar outras características que apresentem viés, também poderá remover esse desequilíbrio. A opção **random_state** foi definida como “42”, mas você pode alterar isso.

In [ ]:
#display-summary
gender = train_data["customer_gender_female"]
gender.value_counts()

In [ ]:
#remove-imbalance
sm = SMOTE(random_state=42)
train_data_upsampled, gender_res = sm.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

Para continuar este laboratório, retorne à <a href="#task4-5-continue" target="_self">Tarefa 4.5</a>.

<a name="task4-6" id="task4-6"></a>

### Apêndice: Retreinar o modelo (Tarefa 4.6)

Você encontrou um desequilíbrio e tem um novo conjunto de dados de treinamento. Use esse conjunto de dados e retreine o arquivo. Para fazer isso, carregue o novo arquivo e crie um estimador. Depois, retreine os dados com **fit()**. Há vários hiperparâmetros de exemplo incluídos no código abaixo. Você pode adicionar, remover ou ajustar esses hiperparâmetros durante o retreinamento para encontrar o melhor modelo.

In [ ]:
#upload-upsampled-csv
train_data_upsampled.to_csv("data/upsampled_train.csv", index=False, header=False)
retrain_path = S3Uploader.upload("data/upsampled_train.csv", "s3://{}/{}".format(bucket, prefix))
retrain_input = TrainingInput(retrain_path, content_type="text/csv")

retrain_data_inputs = {
    "train": retrain_input,
    "validation": validation_input
}

In [ ]:
#create-estimator
hyperparameters= {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "300",
}

xgb_retrained = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
)

In [ ]:
#retrain-upsampled-data
xgb_retrained.fit(
    inputs = retrain_data_inputs
) 

Para continuar este laboratório, retorne à <a href="#task4-6-continue" target="_self">Tarefa 4.6</a>.

<a name="task5-1" id="task5-1"></a>

### Apêndice: Criar um trabalho de transformação em lote (Tarefa 5.1)

Use o estimador de modelo e crie um trabalho de transformação em lote com **transformer**. Defina a estratégia de **MultiRecord** para aumentar a eficiência do processamento. Depois, passe o **test_path** e aguarde a execução da inferência.

In [ ]:
#create-batch-transformer
# Use the retrained model if it exists, otherwise, use the original model
try:
    model = xgb_retrained
except NameError:
    model = xgb

# Create the transformer
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/".format(bucket, prefix)
)

In [ ]:
#run-batch-transform-job
test_data_batch = test_data.drop("fraud", axis=1)
test_data_batch.to_csv("test_data_batch.csv", index=False, header=False)
test_path_batch = S3Uploader.upload("test_data_batch.csv", "s3://{}/{}".format(bucket, prefix))

transformer.transform(test_path_batch, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

Para continuar este laboratório, retorne à <a href="#task5-1-continue" target="_self">Tarefa 5.1</a>.

<a name="task5-2" id="task5-2"></a>

### Apêndice: Visualizar dados de previsão e precisão de um trabalho de transformação em lote (Tarefa 5.2)

Quando o trabalho de transformação em lote for concluído, visualize os dados de previsão armazenados no Amazon S3. Você pode consultar o caminho de saída que definiu no **transformer** e amostrar os dados.

Nessa saída, a previsão **fraud** é acrescentada ao final de cada registro.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
test_data = pd.read_csv("test_data_batch.csv.out")
test_data.head()

Para continuar este laboratório, retorne à <a href="#task5-2-continue" target="_self">Tarefa 5.2</a>.

<a name="task5-3" id="task5-3"></a>

### Apêndice: limpar as instâncias de SageMaker no SageMaker Studio (Tarefa 5.3)

Ao desenvolver modelos no SageMaker Studio, verifique periodicamente se há instâncias que você deseja limpar. Se houver, você poderá fechá-las por meio do SageMaker Studio.

1. Na barra de menus à esquerda, selecione o ícone **Running Terminals and Kernels** (Terminais e kernels em execução) (círculo com um quadrado no meio).

1. Se houver instâncias que ainda estejam abertas, à direita de cada tipo de instância, escolha o ícone **Shut down** (Fechar). 

Você pode visualizar as aplicações que estão em execução em cada instância para confirmar quais deseja fechar.

1. Se uma janela pop-up aparecer, selecione **Shut down all** (Fechar tudo).

1. Selecione o ícone **Refresh List** (Atualizar lista) periodicamente até que a instância não apareça mais na lista. Podem ser necessários de dois a cinco minutos para que a instância feche.

Você não precisa fechar a instância que o notebook **capstone.ipynb** está usando.

Para continuar este laboratório, retorne à <a href="#task5-3-continue" target="_self">Tarefa 5.3</a>.

<a name="task6-1" id="task6-1"></a>

### Apêndice: configurar um pipeline (Tarefa 6.1)

Para criar um pipeline, defina cada etapa do processo do pipeline e execute-o.

Neste exemplo, você cria as seguintes etapas:
- **AutoModelProcess**: uma etapa **Processing** que obtém o arquivo .csv e divide-o em conjuntos de dados de treinamento, teste e validação.
- **AutoHyperParameterTuning**: uma etapa **Tuning** que obtém um intervalo de hiperparâmetros e ajusta o modelo.
- **AutoEvalBestModel**: uma etapa **Processing** que cria um relatório de avaliação para descrever o melhor modelo.
- **CheckAUCScoreAutoEvaluation**: uma etapa **Condition** que avalia os modelos com base em uma métrica de avaliação. 
- **AutoCreateModel**: uma etapa **Model** que cria um modelo.
- **RegisterAutoModel-RegisterModel**: uma etapa **RegisterModel** que registra um modelo.
- **AutoModelConfigFile**: uma etapa **Processing** que cria um relatório de viés.
- **AutoTransform**: uma etapa **Transform** que executa um trabalho de transformação em lote.
- **ClarifyProcessingStep**: uma etapa **Processing** que executa um trabalho do SageMaker Clarify.

Se você não souber como proceder em algum momento durante a criação do pipeline, personalize o código a seguir ou use-o como um guia para criar o pipeline.

In [ ]:
#run-pipeline
# Set the variables
model_name = "Auto-model"
sklearn_processor_version="0.23-1"
model_package_group_name="AutoModelPackageGroup"
pipeline_name= "AutoModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

# Upload files to the default S3 bucket
s3_client.put_object(Bucket=bucket,Key='data/')
s3_client.put_object(Bucket=bucket,Key='input/code/')
s3_client.upload_file(Filename="data/batch_data.csv", Bucket=bucket, Key="data/batch_data.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key="data/claims_customer.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="pipelines/evaluate.py", Bucket=bucket, Key="input/code/evaluate.py")
s3_client.upload_file(Filename="pipelines/generate_config.py", Bucket=bucket, Key="input/code/generate_config.py")
s3_client.upload_file(Filename="pipelines/preprocess.py", Bucket=bucket, Key="input/code/preprocess.py")

# Configure important settings. Change the input_data if you want to
# use a file other than the claims_customer.csv and batch_data.csv files.
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
        name="TrainingInstanceType",
        default_value="ml.m5.xlarge"
)
input_data = ParameterString(
        name="InputData",
        default_value="s3://{}/data/claims_customer.csv".format(bucket), 
)
batch_data = ParameterString(
        name="BatchData",
        default_value="s3://{}/data/batch_data.csv".format(bucket),
)

# Run a scikit-learn script to do data processing on SageMaker using 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version=sklearn_processor_version,
        instance_type=processing_instance_type.default_value, 
        instance_count=processing_instance_count,
        sagemaker_session=sagemaker_session,
        role=role,
)

# Configure the processing step to pull in the input_data
step_process = ProcessingStep(
        name="AutoModelProcess",
        processor=sklearn_processor,
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train",\
                             destination=f"s3://{bucket}/output/train" ),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",\
                            destination=f"s3://{bucket}/output/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test",\
                            destination=f"s3://{bucket}/output/test"),
            ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch",\
                            destination=f"s3://{bucket}/data/batch"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline",\
                            destination=f"s3://{bucket}/input/baseline")
        ],
        code=f"s3://{bucket}/input/code/preprocess.py",
        job_arguments=["--input-data", input_data],
)

# Set up the model path, image uri, and hyperparameters for the estimator
model_path = f"s3://{bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1",
    py_version="py3",
    instance_type=training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=model_path,
    base_job_name=f"auto-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set the hyperparameter ranges for the tuning step and configure the tuning step
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "AutoHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs=2, max_parallel_jobs=2),
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# Configure the processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-auto-eval",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="AutoEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="AutoEvalBestModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",\
                            destination=f"s3://{bucket}/output/evaluation"),
    ],
    code=f"s3://{bucket}/input/code/evaluate.py",
    property_files=[evaluation_report],
)

# Configure model creation
model = Model(
    image_uri=image_uri,        
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name="AutoCreateModel",
    model=model,
    inputs=inputs,
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=clarify_image,
    role=role,
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session,
)

bias_report_output_path = f"s3://{bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
step_config_file = ProcessingStep(
    name="AutoModelConfigFile",
    processor=script_processor,
    code=f"s3://{bucket}/input/code/generate_config.py",
    job_arguments=["--modelname",step_create_model.properties.ModelName,"--bias-report-output-path",bias_report_output_path,"--clarify-instance-type",clarify_instance_type,\
                  "--default-bucket",bucket,"--num-baseline-samples","50","--instance-count","1"],
    depends_on= [step_create_model.name]
)

# Configure the step to perform a batch transform job
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line",
    accept="text/csv",    
    output_path=f"s3://{bucket}/AutoTransform"
)

step_transform = TransformStep(
    name="AutoTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data,content_type="text/csv",join_source="Input",split_type="Line")
)

# Configure the SageMaker Clarify processing step
analysis_config_path = f"s3://{bucket}/clarify-output/bias/analysis_config.json"

data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=f's3://{bucket}/output/train/train.csv', 
    s3_output_path=bias_report_output_path,
    label=0,
    headers=list(pd.read_csv("./data/claims_customer.csv", index_col=None).columns), #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
    dataset_type="text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type=clarify_instance_type,
    sagemaker_session=sagemaker_session,
)

config_input = ProcessingInput(
    input_name="analysis_config",
    source=analysis_config_path,
    destination="/opt/ml/processing/input/analysis_config",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_compression_type="None",
)

data_input = ProcessingInput(
    input_name="dataset",
    source=data_config.s3_data_input_path,
    destination="/opt/ml/processing/input/data",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type=data_config.s3_data_distribution_type,
    s3_compression_type=data_config.s3_compression_type,
)

result_output = ProcessingOutput(
    source="/opt/ml/processing/output",
    destination=data_config.s3_output_path,
    output_name="analysis_result",
    s3_upload_mode="EndOfJob",
)

step_clarify = ProcessingStep(
    name="ClarifyProcessingStep",
    processor=clarify_processor,
    inputs= [data_input, config_input],
    outputs=[result_output],
    depends_on = [step_config_file.name]
)

# Configure the model registration step
model_statistics = MetricsSource(
    s3_uri="s3://{}/output/evaluation/evaluation.json".format(bucket),
    content_type="application/json"
)
explainability = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
)

bias = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
) 

model_metrics = ModelMetrics(
    model_statistics=model_statistics,
    explainability=explainability,
    bias=bias
)

step_register = RegisterModel(
    name="RegisterAutoModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
)

# Create the model evaluation step
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value"
    ),
    right=0.75,
)

step_cond = ConditionStep(
    name="CheckAUCScoreAutoEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model,step_config_file,step_transform,step_clarify,step_register],
    else_steps=[],
)

# Define the pipeline
def get_pipeline(
    region,
    role=None,
    default_bucket=None,
    model_package_group_name="AutoModelPackageGroup",
    pipeline_name="AutoModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version=None
    ):
    """Gets a SageMaker ML Pipeline instance working with auto data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """

    # pipeline instance
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps=[step_process,step_tuning,step_eval,step_cond],
        sagemaker_session=sagemaker_session
    )
    return pipeline


# Create the pipeline
pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

pipeline.upsert(role_arn=role)

# Run the pipeline
RunPipeline = pipeline.start()

Para continuar este laboratório, retorne à <a href="#task6-1-continue" target="_self">Tarefa 6.1</a>.

<a name="task6-2" id="task6-2"></a>

### Apêndice: monitorar um pipeline (Tarefa 6.2)

Agora que você já criou e executou o pipeline, monitore-o. Veja o status do pipeline no SageMaker Studio.

Se você quiser excluir o pipeline, remova-o usando **delete_pipeline**.

In [ ]:
#describe-pipeline
RunPipeline.describe()

In [ ]:
#list-pipeline-steps
RunPipeline.list_steps()

In [ ]:
#remove-pipeline
response = sagemaker_session.boto_session.client("sagemaker", region_name=region).delete_pipeline(PipelineName='AutoModelSMPipeline')
print(response)

Para continuar este laboratório, retorne à <a href="#task6-2-continue" target="_self">Tarefa 6.2</a>.